### Packer


The ``pp.pack()`` function is able to pack geometries together into rectangular bins. If a ``max_size`` is specified, the function will create as many bins as is necessary to pack all the geometries and then return a list of the filled-bin Components. The function is based on PHIDL as well as this tutorial.

Here we generate several random shapes then pack them together automatically. We allow the bin to be as large as needed to fit all the Devices by specifying ``max_size = (None, None)``.  By setting ``aspect_ratio = (2,1)``, we specify the rectangular bin it tries to pack them into should be twice as wide as it is tall:


In [ ]:
import numpy as np
import pp
from pp import qp

np.random.seed(5)
D_list = [pp.c.ellipse(radii = np.random.rand(2)*n+2) for n in range(50)]
D_list += [pp.c.rectangle(size = np.random.rand(2)*n+2) for n in range(50)]

D_packed_list = pp.pack(
        D_list,                 # Must be a list or tuple of Component
        spacing = 1.25,         # Minimum distance between adjacent shapes
        aspect_ratio = (2,1),   # (width, height) ratio of the rectangular bin
        max_size = (None,None), # Limits the size into which the shapes will be packed
        density = 1.05,          # Values closer to 1 pack tighter but require more computation
        sort_by_area = True,    # Pre-sorts the shapes by area
        verbose = False,
        )
D = D_packed_list[0] # Only one bin was created, so we plot that
qp(D) # quickplot the geometry

Say we need to pack many shapes into multiple 500x500 unit die. If we set
``max_size = (500,500)`` the shapes will be packed into as many 500x500 unit die
as required to fit them all:

In [ ]:

np.random.seed(1)
D_list = [pp.c.ellipse(radii = np.random.rand(2)*n+2) for n in range(120)]
D_list += [pp.c.rectangle(size = np.random.rand(2)*n+2) for n in range(120)]

D_packed_list = pp.pack(
        D_list,                 # Must be a list or tuple of Component
        spacing = 4,         # Minimum distance between adjacent shapes
        aspect_ratio = (1,1),   # Shape of the box
        max_size = (500,500),   # Limits the size into which the shapes will be packed
        density = 1.05,         # Values closer to 1 pack tighter but require more computation
        sort_by_area = True,    # Pre-sorts the shapes by area
        verbose = False,
        )

# Put all packed bins into a single Component and spread them out with distribute()
F = pp.Component()
[F.add_ref(D) for D in D_packed_list]
F.distribute(elements = 'all', direction = 'x', spacing = 100, separation = True)
qp(F)

Note that the packing problem is an NP-complete problem, so ``pp.pack()`` may
be slow if there are more than a few hundred Components to pack (in that case, try
pre-packing a few dozen at a time then packing the resulting bins). Requires the
``rectpack`` python package.

### Distribute


The ``distribute()`` function allows you to space out elements within a Component
evenly in the x or y direction.  It is meant to duplicate the distribute
functionality present in Inkscape / Adobe Illustrator:

Say we start out with a few random-sized rectangles we want to space out:

In [ ]:

D = pp.Component()
# Create different-sized rectangles and add them to D
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]

qp(D) # quickplot the geometry

Oftentimes, we want to guarantee some distance between the objects.  By setting
``separation = True`` we move each object such that there is ``spacing``
distance between them:

In [ ]:

D = pp.Component()
# Create different-sized rectangles and add them to D
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
# Distribute all the rectangles in D along the x-direction with a separation of 5
D.distribute(elements = 'all',   # either 'all' or a list of objects
             direction = 'x',    # 'x' or 'y'
             spacing = 5,
             separation = True)

qp(D) # quickplot the geometry

Alternatively, we can spread them out on a fixed grid by setting ``separation =
False``. Here we align the left edge (``edge = 'min'``) of each object along a
grid spacing of 100:

In [ ]:

D = pp.Component()
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 100, separation = False,
             edge = 'xmin') # edge must be either 'xmin' (left), 'xmax' (right), or 'x' (center)

qp(D) # quickplot the geometry

The alignment can be done along the right edge as well by setting ``edge =
'max'``, or along the center by setting ``edge = 'center'`` like in the
following:

In [ ]:

D = pp.Component()
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n-10,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 100, separation = False,
             edge = 'x') # edge must be either 'xmin' (left), 'xmax' (right), or 'x' (center)

qp(D) # quickplot the geometry

### Align


The ``align()`` function allows you to elements within a Component horizontally or
vertically.  It is meant to duplicate the alignment functionality present in
Inkscape / Adobe Illustrator:

Say we ``distribute()`` a few objects, but they're all misaligned:

In [ ]:

D = pp.Component()
# Create different-sized rectangles and add them to D then distribute them
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 5, separation = True)

qp(D) # quickplot the geometry

we can use the ``align()`` function to align their top edges (``alignment =
'ymax'):

In [ ]:

D = pp.Component()
# Create different-sized rectangles and add them to D then distribute them
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 5, separation = True)

# Align top edges
D.align(elements = 'all', alignment = 'ymax')

qp(D) # quickplot the geometry

or align their centers (``alignment = 'y'):

In [ ]:
D = pp.Component()
# Create different-sized rectangles and add them to D then distribute them
[D.add_ref(pp.c.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 5, separation = True)

# Align top edges
D.align(elements = 'all', alignment = 'y')

qp(D) # quickplot the geometry

other valid alignment options include ``'xmin', 'x', 'xmax', 'ymin', 'y', and
'ymax'``